In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
# spark = (SparkSession.builder
#   .appName("SparkSessionExample")
#   .config('spark.jars.packages', 'org.mariadb.jdbc:mariadb-java-client:2.7.4,org.apache.spark:spark-sql_2.12:3.1.2,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2')
#   .master("spark://spark:7077").getOrCreate())

spark = (SparkSession.builder
  .appName("SparkSessionExample")
  .config('spark.jars.packages', 'mysql:mysql-connector-java:8.0.26,org.apache.spark:spark-sql_2.12:3.1.2,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2')
  .master("spark://spark:7077").getOrCreate())

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
mysql#mysql-connector-java added as a dependency
org.apache.spark#spark-sql_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5461612f-bc56-49d0-9882-ac3766270078;1.0
	confs: [default]
	found mysql#mysql-connector-java;8.0.26 in central
	found com.google.protobuf#protobuf-java;3.11.4 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-p

In [3]:
# df = sqlContext.read.format("jdbc").options(
#     url="jdbc:mariadb://mariadb:3306/data-warehouse",
#     driver="org.mariadb.jdbc.Driver",
#     dbtable="transactions_subset",
#     user="root",
#     properties={"driver": 'org.mariadb.jdbc.Driver'}
# ).load()

# df = spark.read\
#     .format("jdbc")\
#     .option("url", "jdbc:mariadb://mariadb:3306/backup")\
#     .option("driver", "org.mariadb.jdbc.Driver")\
#     .option("query", "SELECT manufacturer FROM catalog LIMIT 2")\
#     .option("user", "root").load()


df = spark.read\
    .format("jdbc")\
    .option("url", "jdbc:mysql://mariadb:3306/backup")\
    .option("driver", "com.mysql.cj.jdbc.Driver")\
    .option("query", "SELECT * FROM catalog")\
    .option("user", "root").load()

df.show()

+------------+--------------------+--------+--------------+--------+----------+
|manufacturer|        product_name|    size|     item_type|     sku|base_price|
+------------+--------------------+--------+--------------+--------+----------+
|   Zatarains|  Jambalaya Rice Mix|   12 oz| RICE/RICE MIX|42081001|      2.49|
|   Zatarains|  Jambalaya Rice Mix|    8 oz| RICE/RICE MIX|42082001|      1.79|
|     Yucatan|   Guacamole Regular|    8 oz|              |42083001|      3.99|
|       Yuban|Coffee Original B...|   12 oz|COFFEE/CREAMER|42084001|      3.99|
|     Yoplait| GoGurt Variety Pack|    8 ct|        YOGURT|42085001|      2.99|
|    Wishbone|    Italian Dressing|   16 oz|SALAD DRESSING|42086001|      2.00|
|White Castle|Cheeseburger Heat...|29.28 oz|              |42087001|     11.59|
|     Whiskas| Choice Cuts Poultry|   36 oz|      PET FOOD|42088001|      4.99|
|      Welchs|Farmers Pick Conc...|   46 oz|              |42089001|      3.59|
|      Welchs|     Juice Red Grape|   64

In [4]:
df.select(F.col('sku').cast('string').alias('key'), F.to_json(F.struct("*")).alias("value"))\
    .write \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("topic", "spark") \
    .save()